In [8]:
import requests, json
from audioRecorderAutoStop import record_to_file
from bingSpeech.speechToText import stt
from bingSpeech.textToSpeech import tts
from pprint import pprint
from pymongo import MongoClient

In [10]:
def getDBCreds():
    credentials = json.load(open('../credentials.json'))
    dbCreds = credentials['database2']
    return dbCreds

In [11]:
def connectToDB(dbCreds):
    mongoConnString = "mongodb://" + dbCreds['username'] + ":" + dbCreds['password'] + "@" + dbCreds['host'] + ":" + str(dbCreds['port']) + '/' + dbCreds['dbName']
#     mongoConnString = "mongodb://{0}:{1}@{2}:{3}/{4}".format(dbCreds['username'], dbCreds['password'], dbCreds['host'], str(dbCreds['port']), dbCreds['dbName'])
    client = MongoClient(mongoConnString)
    return client

In [12]:
def getClient():
    dbCreds = getDBCreds()
    client = connectToDB(dbCreds)
    return client

In [6]:
def getDocCollection(client):
    db = client["mitcsy"]
    docCollection = db["doctors"]
    return docCollection

In [40]:
def getMailID(name):
    client = getClient()
    docCollection = getDocCollection(client)
    record = docCollection.find_one(filter={'name':name})
    if(record):
        return(record['mail'])
    else:
        return(False)

In [42]:
a = getMailID('foster')

In [45]:
def sendMail(data):
    url = "INVALID URL"
    auth=("api", "INVALID API KEY")
    response = requests.post(url=url, auth=auth, data=data)
    if(response.status_code == 200):
        print("Success!")
        return("Successfully sent mail")
    else:
        return("Failed to send mail. Try again later")

In [50]:
def getMailDetails():
    tts("What shall I enter as the subject?")
    record_to_file('mailSubject.wav')
    sub = stt('mailSubject.wav')
    if(sub[0] == False):
        print("Note: Error in STT while obtaining subject")
        sub = False
        return((False, False, False))
    tts('Whom should the mail be sent to?')
    record_to_file('mailTo.wav')
    mailTo = stt('mailTo.wav')
    if(mailTo[0] == False):
        print("Error in STT while obtaining email address to send to.")
        mailTo = False
        return((False, False, False))
    tts('What should be in the body?')
    record_to_file('mailBody.wav')
    body = stt('mailBody.wav')
    if(body[0] == False):
        print("Error in STT while obtaining body of mail")
        body = False
        return((False, False, False))
    mailDetails = (mailTo[1], sub[1], body[1])
    return(mailDetails)

In [46]:
def createMail():
    toMail, subject, body = getMailDetails()
    mailId = getMailID(mailTo)
    if(mailId == False):
        print("Mail ID not found")
        return(False)
    fromMail = "Mailgun Sandbox <postmaster@sandbox62818be204d9434b94235950b924fb19.mailgun.org>"
    data = {
        "from" : fromMail,
        "to" : toMail,
        "subject" : subject,
        "text" : body
    }
    print("Data is: ")
    pprint(data)
    return(data)

In [41]:
def mailGun():
    data = createMail()
    if(data == False):
        print("Unable to send mail. Unable to obtain mail address")
        return("Unable to send mail. Unable to obtain mail address")
    response = sendMail(data)
    return(response)